# Lab 3.3: Knowledge Distillation - Training

**Goal:** Implement knowledge distillation training to transfer knowledge from teacher to student.

**You will learn to:**
- Implement Hinton's distillation loss (CE + KL divergence)
- Implement MiniLM's self-attention distillation
- Configure training with temperature scaling
- Monitor training progress and validation metrics
- Save distilled student model

---

## Knowledge Distillation Training Flow

```mermaid
graph LR
    A[Input Batch] --> B[Teacher Forward]
    A --> C[Student Forward]
    
    B --> D[Teacher Logits<br/>+ Attentions]
    C --> E[Student Logits<br/>+ Attentions]
    
    D --> F[Loss Calculation]
    E --> F
    A --> F
    
    F --> G[L_CE: Hard Labels]
    F --> H[L_KD: Soft Labels]
    F --> I[L_attn: Attention]
    
    G --> J[Weighted Sum]
    H --> J
    I --> J
    
    J --> K[Backward Pass]
    K --> L[Update Student]
```

---

## Prerequisites

Make sure you have completed **01-Setup.ipynb** and have:
- `teacher_model`: BERT-base teacher model
- `student_model`: BERT-6L student model
- `tokenizer`: BERT tokenizer
- `tokenized_train`: Training dataset
- `tokenized_val`: Validation dataset

---
## Step 1: Verify Prerequisites

Check that all required components are available.

In [ ]:
import torch
import torch.nn.functional as F

print("=" * 60)
print("Prerequisites Check")
print("=" * 60)

# Check required variables
try:
    assert 'teacher_model' in dir(), "teacher_model not found. Run 01-Setup.ipynb first!"
    assert 'student_model' in dir(), "student_model not found. Run 01-Setup.ipynb first!"
    assert 'tokenizer' in dir(), "tokenizer not found. Run 01-Setup.ipynb first!"
    assert 'tokenized_train' in dir(), "tokenized_train not found. Run 01-Setup.ipynb first!"
    assert 'tokenized_val' in dir(), "tokenized_val not found. Run 01-Setup.ipynb first!"
    
    print("✅ Teacher model available")
    print("✅ Student model available")
    print("✅ Tokenizer available")
    print(f"✅ Training data: {len(tokenized_train)} samples")
    print(f"✅ Validation data: {len(tokenized_val)} samples")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n✅ Device: {device}")
    
    print("\n✅ All prerequisites satisfied!")
    
except AssertionError as e:
    print(f"❌ {e}")
    print("\nPlease run 01-Setup.ipynb first to set up the environment.")

print("=" * 60)

---
## Step 2: Define Distillation Loss Functions

Implement Hinton's KD loss and MiniLM attention loss.

In [ ]:
import torch.nn as nn

def distillation_loss(
    student_logits,
    teacher_logits,
    labels,
    temperature=4.0,
    alpha=0.1
):
    """
    Hinton's Knowledge Distillation Loss.
    
    L_total = α · L_CE(student, hard_labels) + (1-α) · T² · L_KL(student, teacher)
    
    Args:
        student_logits: Student model output logits [batch, num_classes]
        teacher_logits: Teacher model output logits [batch, num_classes]
        labels: Ground truth labels [batch]
        temperature: Temperature for softening probabilities
        alpha: Weight for hard label loss (1-alpha for soft label)
    
    Returns:
        total_loss, ce_loss, kd_loss
    """
    # Hard label loss (Cross-Entropy with T=1)
    ce_loss = F.cross_entropy(student_logits, labels)
    
    # Soft label loss (KL Divergence with temperature)
    # Soften teacher and student logits
    soft_teacher = F.softmax(teacher_logits / temperature, dim=-1)
    soft_student = F.log_softmax(student_logits / temperature, dim=-1)
    
    # KL divergence (multiply by T² to balance gradients)
    kd_loss = F.kl_div(
        soft_student,
        soft_teacher,
        reduction='batchmean'
    ) * (temperature ** 2)
    
    # Total loss (weighted sum)
    total_loss = alpha * ce_loss + (1 - alpha) * kd_loss
    
    return total_loss, ce_loss, kd_loss


def attention_distillation_loss(student_attentions, teacher_attentions):
    """
    MiniLM Self-Attention Distillation Loss.
    
    L_attn = MSE(student_attention, teacher_attention)
    
    Args:
        student_attentions: List of attention matrices per layer
                           Each: [batch, heads, seq_len, seq_len]
        teacher_attentions: List of teacher attention matrices
    
    Returns:
        attention_loss
    """
    loss = 0.0
    num_layers = len(student_attentions)
    
    # Map teacher layers to student layers (隔層對齊)
    # Student layer i corresponds to teacher layer 2i
    teacher_layer_mapping = [0, 2, 4, 6, 8, 10]
    
    for student_idx, teacher_idx in enumerate(teacher_layer_mapping):
        if student_idx >= num_layers:
            break
        
        student_attn = student_attentions[student_idx]
        teacher_attn = teacher_attentions[teacher_idx]
        
        # Average over attention heads
        student_attn_mean = student_attn.mean(dim=1)  # [batch, seq_len, seq_len]
        teacher_attn_mean = teacher_attn.mean(dim=1)
        
        # MSE loss
        loss += F.mse_loss(student_attn_mean, teacher_attn_mean)
    
    return loss / num_layers


print("=" * 60)
print("Distillation Loss Functions Defined")
print("=" * 60)
print("\n✅ Functions available:")
print("   1. distillation_loss(): Hinton's KD (CE + KL)")
print("   2. attention_distillation_loss(): MiniLM attention")
print("\n📝 Loss Formula:")
print("   L_total = α·L_CE + β·L_KD + γ·L_attn")
print("=" * 60)

---
## Step 3: Configure Training Hyperparameters

Set distillation and training parameters.

In [ ]:
# Distillation hyperparameters
TEMPERATURE = 4.0       # Temperature for softening (Hinton recommends 3-10)
ALPHA = 0.1             # Weight for hard label loss
BETA = 0.5              # Weight for soft label loss (KL divergence)
GAMMA = 0.4             # Weight for attention loss

# Training hyperparameters
LEARNING_RATE = 2e-5    # Adam learning rate
BATCH_SIZE = 32         # Training batch size
NUM_EPOCHS = 3          # Number of training epochs
WARMUP_STEPS = 500      # Learning rate warmup
WEIGHT_DECAY = 0.01     # L2 regularization
MAX_GRAD_NORM = 1.0     # Gradient clipping

# Logging
LOG_STEPS = 100         # Log every N steps
EVAL_STEPS = 500        # Evaluate every N steps

print("=" * 60)
print("Training Configuration")
print("=" * 60)

print("\n📊 Distillation Parameters:")
print(f"   Temperature (T): {TEMPERATURE}")
print(f"   Alpha (hard label): {ALPHA}")
print(f"   Beta (soft label): {BETA}")
print(f"   Gamma (attention): {GAMMA}")
print(f"   Weight sum: {ALPHA + BETA + GAMMA}")

print("\n📊 Training Parameters:")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {NUM_EPOCHS}")
print(f"   Warmup steps: {WARMUP_STEPS}")
print(f"   Weight decay: {WEIGHT_DECAY}")

# Calculate training steps
num_training_steps = (len(tokenized_train) // BATCH_SIZE) * NUM_EPOCHS
print(f"\n📊 Training Schedule:")
print(f"   Total steps: {num_training_steps}")
print(f"   Steps per epoch: {len(tokenized_train) // BATCH_SIZE}")
print(f"   Warmup steps: {WARMUP_STEPS}")

print("=" * 60)

---
## Step 4: Prepare Data Loaders

Create training and validation data loaders.

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

print("=" * 60)
print("Preparing Data Loaders")
print("=" * 60)

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Set format for PyTorch
tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_val.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Create data loaders
train_dataloader = DataLoader(
    tokenized_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=data_collator
)

val_dataloader = DataLoader(
    tokenized_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=data_collator
)

print(f"\n✅ Data loaders created:")
print(f"   Train batches: {len(train_dataloader)}")
print(f"   Val batches: {len(val_dataloader)}")
print(f"   Batch size: {BATCH_SIZE}")

# Test data loader
sample_batch = next(iter(train_dataloader))
print(f"\n📝 Sample Batch:")
print(f"   Input IDs shape: {sample_batch['input_ids'].shape}")
print(f"   Attention mask shape: {sample_batch['attention_mask'].shape}")
print(f"   Labels shape: {sample_batch['labels'].shape}")

print("=" * 60)

---
## Step 5: Setup Optimizer and Scheduler

Configure AdamW optimizer with linear warmup.

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

print("=" * 60)
print("Setting up Optimizer and Scheduler")
print("=" * 60)

# Optimizer (only update student parameters)
optimizer = AdamW(
    student_model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)

# Learning rate scheduler with warmup
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=num_training_steps
)

print("\n✅ Optimizer: AdamW")
print(f"   Learning rate: {LEARNING_RATE}")
print(f"   Weight decay: {WEIGHT_DECAY}")
print(f"   Parameters to optimize: {sum(p.numel() for p in student_model.parameters()):,}")

print("\n✅ Scheduler: Linear with warmup")
print(f"   Warmup steps: {WARMUP_STEPS}")
print(f"   Total steps: {num_training_steps}")

print("=" * 60)

---
## Step 6: Training Loop Implementation

Implement the main distillation training loop.

In [ ]:
from tqdm import tqdm
import time
import numpy as np

print("=" * 60)
print("Starting Knowledge Distillation Training")
print("=" * 60)

# Training history
history = {
    'train_loss': [],
    'train_ce_loss': [],
    'train_kd_loss': [],
    'train_attn_loss': [],
    'val_loss': [],
    'val_accuracy': [],
    'learning_rate': []
}

# Freeze teacher model
teacher_model.eval()
for param in teacher_model.parameters():
    param.requires_grad = False

print("\n✅ Teacher model frozen (no gradient updates)")

# Training function
def train_epoch(epoch):
    student_model.train()
    
    total_loss = 0
    total_ce_loss = 0
    total_kd_loss = 0
    total_attn_loss = 0
    
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    
    for step, batch in enumerate(progress_bar):
        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Teacher forward pass (no gradients)
        with torch.no_grad():
            teacher_outputs = teacher_model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                output_attentions=True
            )
            teacher_logits = teacher_outputs.logits
            teacher_attentions = teacher_outputs.attentions
        
        # Student forward pass
        student_outputs = student_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_attentions=True
        )
        student_logits = student_outputs.logits
        student_attentions = student_outputs.attentions
        
        # Calculate losses
        # 1. Hinton's distillation loss (CE + KL)
        dist_loss, ce_loss, kd_loss = distillation_loss(
            student_logits,
            teacher_logits,
            labels,
            temperature=TEMPERATURE,
            alpha=ALPHA
        )
        
        # 2. Attention distillation loss
        attn_loss = attention_distillation_loss(
            student_attentions,
            teacher_attentions
        )
        
        # 3. Total loss (weighted sum)
        loss = BETA * dist_loss + GAMMA * attn_loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(student_model.parameters(), MAX_GRAD_NORM)
        
        # Optimizer step
        optimizer.step()
        scheduler.step()
        
        # Accumulate losses
        total_loss += loss.item()
        total_ce_loss += ce_loss.item()
        total_kd_loss += kd_loss.item()
        total_attn_loss += attn_loss.item()
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'ce': f'{ce_loss.item():.4f}',
            'kd': f'{kd_loss.item():.4f}',
            'attn': f'{attn_loss.item():.4f}',
            'lr': f'{scheduler.get_last_lr()[0]:.2e}'
        })
        
        # Logging
        if (step + 1) % LOG_STEPS == 0:
            avg_loss = total_loss / (step + 1)
            avg_ce = total_ce_loss / (step + 1)
            avg_kd = total_kd_loss / (step + 1)
            avg_attn = total_attn_loss / (step + 1)
            
            history['train_loss'].append(avg_loss)
            history['train_ce_loss'].append(avg_ce)
            history['train_kd_loss'].append(avg_kd)
            history['train_attn_loss'].append(avg_attn)
            history['learning_rate'].append(scheduler.get_last_lr()[0])
    
    return total_loss / len(train_dataloader)


# Validation function
def validate():
    student_model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Validating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Student forward
            outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            
            # Loss
            loss = F.cross_entropy(logits, labels)
            total_loss += loss.item()
            
            # Predictions
            preds = torch.argmax(logits, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    avg_loss = total_loss / len(val_dataloader)
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    
    history['val_loss'].append(avg_loss)
    history['val_accuracy'].append(accuracy)
    
    return avg_loss, accuracy


# Main training loop
print("\n" + "=" * 60)
print("Training Progress")
print("=" * 60)

best_val_accuracy = 0.0
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"{'='*60}")
    
    # Training
    train_loss = train_epoch(epoch)
    
    # Validation
    val_loss, val_accuracy = validate()
    
    # Print epoch summary
    print(f"\n📊 Epoch {epoch+1} Summary:")
    print(f"   Train Loss: {train_loss:.4f}")
    print(f"   Val Loss:   {val_loss:.4f}")
    print(f"   Val Acc:    {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
    
    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        print(f"   ✅ New best accuracy! Saving model...")
        # We'll save in the next step

total_time = time.time() - start_time

print("\n" + "=" * 60)
print("Training Complete!")
print("=" * 60)
print(f"\n📊 Final Results:")
print(f"   Total time: {total_time/60:.2f} minutes")
print(f"   Best val accuracy: {best_val_accuracy:.4f} ({best_val_accuracy*100:.2f}%)")
print("=" * 60)

---
## Step 7: Visualize Training Progress

Plot training curves to analyze convergence.

In [ ]:
import matplotlib.pyplot as plt

print("=" * 60)
print("Visualizing Training Progress")
print("=" * 60)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Total Loss
ax = axes[0, 0]
if len(history['train_loss']) > 0:
    ax.plot(history['train_loss'], label='Train Loss', color='blue', alpha=0.7)
ax.set_xlabel('Steps (x100)', fontsize=11)
ax.set_ylabel('Loss', fontsize=11)
ax.set_title('Training Loss', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Loss Components
ax = axes[0, 1]
if len(history['train_ce_loss']) > 0:
    ax.plot(history['train_ce_loss'], label='CE Loss (hard)', color='green', alpha=0.7)
    ax.plot(history['train_kd_loss'], label='KD Loss (soft)', color='orange', alpha=0.7)
    ax.plot(history['train_attn_loss'], label='Attn Loss', color='purple', alpha=0.7)
ax.set_xlabel('Steps (x100)', fontsize=11)
ax.set_ylabel('Loss', fontsize=11)
ax.set_title('Loss Components', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Validation Accuracy
ax = axes[1, 0]
if len(history['val_accuracy']) > 0:
    epochs_range = range(1, len(history['val_accuracy']) + 1)
    ax.plot(epochs_range, history['val_accuracy'], 'o-', 
            color='red', linewidth=2, markersize=8, label='Val Accuracy')
    ax.axhline(y=best_val_accuracy, color='green', linestyle='--', 
               label=f'Best: {best_val_accuracy:.4f}', linewidth=2)
ax.set_xlabel('Epoch', fontsize=11)
ax.set_ylabel('Accuracy', fontsize=11)
ax.set_title('Validation Accuracy', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Learning Rate Schedule
ax = axes[1, 1]
if len(history['learning_rate']) > 0:
    ax.plot(history['learning_rate'], color='purple', alpha=0.7)
ax.set_xlabel('Steps (x100)', fontsize=11)
ax.set_ylabel('Learning Rate', fontsize=11)
ax.set_title('Learning Rate Schedule', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('./distillation_training.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Training curves saved to distillation_training.png")
print("=" * 60)

---
## Step 8: Save Distilled Student Model

Save the trained student model for inference.

In [ ]:
import os
import json

# Output directory
OUTPUT_DIR = "./distilled_student"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("=" * 60)
print("Saving Distilled Student Model")
print("=" * 60)
print(f"Output directory: {OUTPUT_DIR}\n")

# Save model
print("⏳ Saving model...")
student_model.save_pretrained(OUTPUT_DIR)
print("✅ Model saved")

# Save tokenizer
print("⏳ Saving tokenizer...")
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ Tokenizer saved")

# Save training configuration
distillation_config = {
    "method": "knowledge_distillation",
    "teacher_model": "bert-base-uncased",
    "student_layers": 6,
    "compression_ratio": 2.1,
    "temperature": TEMPERATURE,
    "alpha": ALPHA,
    "beta": BETA,
    "gamma": GAMMA,
    "learning_rate": LEARNING_RATE,
    "batch_size": BATCH_SIZE,
    "num_epochs": NUM_EPOCHS,
    "best_val_accuracy": best_val_accuracy,
    "training_time_minutes": total_time / 60
}

config_path = os.path.join(OUTPUT_DIR, "distillation_config.json")
with open(config_path, 'w') as f:
    json.dump(distillation_config, f, indent=2)

print(f"✅ Config saved to {config_path}")

# Save training history
history_path = os.path.join(OUTPUT_DIR, "training_history.json")
with open(history_path, 'w') as f:
    json.dump(history, f, indent=2)

print(f"✅ History saved to {history_path}")

# List saved files
saved_files = os.listdir(OUTPUT_DIR)
print(f"\n📁 Saved files ({len(saved_files)}):")
for file in sorted(saved_files)[:8]:
    file_path = os.path.join(OUTPUT_DIR, file)
    if os.path.isfile(file_path):
        size_mb = os.path.getsize(file_path) / 1e6
        print(f"   {file:40s} {size_mb:>10.2f} MB")

if len(saved_files) > 8:
    print(f"   ... and {len(saved_files) - 8} more files")

print("\n" + "=" * 60)
print("✅ Student model saved successfully!")
print("=" * 60)

---
## Step 9: Compare Pre/Post Distillation Performance

Evaluate improvement from distillation.

In [ ]:
print("=" * 60)
print("Pre vs Post Distillation Comparison")
print("=" * 60)

# Recall pre-distillation accuracy (if available)
if 'student_accuracy' in dir():
    pre_distill_acc = student_accuracy
else:
    pre_distill_acc = 0.85  # Typical baseline

# Recall teacher accuracy
if 'teacher_accuracy' in dir():
    teacher_acc = teacher_accuracy
else:
    teacher_acc = 0.928  # Typical BERT-base on SST-2

post_distill_acc = best_val_accuracy

print("\n📊 Performance Summary:\n")
print(f"   {'Model':<30} {'Accuracy':<15} {'Relative to Teacher'}")
print(f"   {'-'*30} {'-'*15} {'-'*20}")
print(f"   {'Teacher (BERT-base)':<30} {teacher_acc:.4f} ({teacher_acc*100:.2f}%)  100.0%")
print(f"   {'Student (Pre-distillation)':<30} {pre_distill_acc:.4f} ({pre_distill_acc*100:.2f}%)  {pre_distill_acc/teacher_acc*100:.1f}%")
print(f"   {'Student (Post-distillation)':<30} {post_distill_acc:.4f} ({post_distill_acc*100:.2f}%)  {post_distill_acc/teacher_acc*100:.1f}%")

improvement = (post_distill_acc - pre_distill_acc) * 100
gap_closed = (post_distill_acc - pre_distill_acc) / (teacher_acc - pre_distill_acc) * 100

print(f"\n📈 Distillation Impact:")
print(f"   Absolute improvement: +{improvement:.2f}%")
print(f"   Gap closed: {gap_closed:.1f}%")
print(f"   Remaining gap: {(teacher_acc - post_distill_acc)*100:.2f}%")

if post_distill_acc / teacher_acc >= 0.98:
    print("\n✅ Excellent! Student achieves >=98% of teacher performance.")
elif post_distill_acc / teacher_acc >= 0.95:
    print("\n✅ Good! Student achieves 95-98% of teacher performance.")
elif post_distill_acc / teacher_acc >= 0.90:
    print("\n🟡 Acceptable. Student achieves 90-95% of teacher performance.")
else:
    print("\n⚠️  Student performance is <90% of teacher. Consider:")
    print("   - Increasing training epochs")
    print("   - Adjusting temperature (try T=3 or T=5)")
    print("   - Increasing student model capacity")

print("=" * 60)

---
## ✅ Distillation Training Complete!

**Summary**:
- ✅ Implemented Hinton's distillation loss (CE + KL divergence)
- ✅ Implemented MiniLM attention distillation
- ✅ Trained for 3 epochs with temperature scaling
- ✅ Monitored training progress and validation metrics
- ✅ Saved distilled student model
- ✅ Achieved significant performance improvement

**Key Results**:
- **Pre-distillation**: ~85-88% accuracy (layer-wise initialized)
- **Post-distillation**: ~91-92% accuracy
- **Teacher accuracy**: ~92.8%
- **Relative performance**: 98-99% of teacher
- **Compression ratio**: 2.1x (110M → 52M params)

**Training Details**:
- Temperature: 4.0
- Loss weights: α=0.1 (hard), β=0.5 (soft), γ=0.4 (attention)
- Training time: ~30-45 minutes (3 epochs)
- Best validation accuracy saved

**Next Steps**:
1. Proceed to **03-Inference.ipynb** for detailed quality evaluation
2. Compare teacher vs student outputs side-by-side
3. Analyze inference speed improvements

**Key Variables Available**:
- `student_model`: Distilled student model
- `history`: Training history (loss curves, accuracy)
- `best_val_accuracy`: Best validation accuracy achieved
- `OUTPUT_DIR`: Path to saved model

---

**⏭️ Continue to**: [03-Inference.ipynb](./03-Inference.ipynb)